In [ ]:
# -*- coding: utf-8 -*-
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
# implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

# **Always run all imported notebooks when you make a change in some files you imported.**

### Adam Candrák/Mária Matušisková - 50%/50%

# Imports

In [ ]:
import import_ipynb
import connections
# import devices
# import normalize
import processes
# import profiles

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
from collections import Counter
from sklearn.model_selection import train_test_split


Just for test purposes to check if the import of jupyter notebooks was successful.

In [ ]:
connections.test()
# devices.test()
processes.test()
# profiles.test()

# Phase 1 - Exploratory analysis

-------------------------------------------------------------------------------------------

## 2.1 Data Preparation

Change the names of columns of connections dataset:

In [ ]:
c_connections = connections.connections.rename(columns={
    "c.katana": "facebook",
    "c.android.chrome": "chrome",
    "c.android.gm": "gmail",
    "c.dogalize": "dogalize",
    "c.android.youtube": "youtube",
    "c.updateassist": "updateassist",
    "c.UCMobile.intl": "UCMobile.intl",
    "c.raider": "raider",
    "c.android.vending": "vending",
    "c.UCMobile.x86": "UCMobile.x86",
})

Change the names of columns of processes dataset:

In [ ]:
p_processes = processes.processes.rename(columns={
    "p.katana": "facebook",
    "p.android.chrome": "chrome",
    "p.android.gm": "gmail",
    "p.dogalize": "dogalize",
    "p.android.vending": "vending",
    "p.android.packageinstaller": "packageinstaller",
    "p.system": "system",
    "p.android.documentsui": "documentsui",
    "p.android.settings": "settings",
    "p.android.externalstorage": "externalstorage",
    "p.android.defcontainer": "defcontainer",
    "p.inputmethod.latin": "inputmethod.latin",
    "p.process.gapps": "gapps",
    "p.simulator": "simulator",
    "p.android.gms": "google mobile services (gms)",
    "p.google": "google",
    "p.olauncher": "olauncher",
    "p.browser.provider": "browser provider",
    "p.notifier": "notifier",
    "p.gms.persistent": "gms.persistent",
})

Change type of timestamp to int64 of connections dataset:

In [ ]:
c_connections['ts'] = pd.to_datetime(c_connections['ts']).astype(np.int64)

Change type of timestamp to int64 of processes dataset:

In [ ]:
p_processes['ts'] = pd.to_datetime(p_processes['ts']).astype(np.int64)

#### Merge datasets connections and processes:

In [ ]:
new_dataset = pd.merge(c_connections, p_processes, on=['imei', 'ts', 'mwra'])
new_dataset.head()

In [ ]:
new_dataset

#### Data cleaning:

Find negative values in the merged dataset:

In [ ]:
negative_values = new_dataset.select_dtypes(include=[np.number]) < 0

# any for columns and all values in the series of the first any
has_negatives = negative_values.any().any()

if has_negatives:
    print("The dataset has negative values.")
    print(negative_values.any())  
else:
    print("No negative values found in the dataset.")

Find NaN values in the merged dataset:

In [ ]:
has_nan = new_dataset.isnull().values.any()

if has_nan:
    print("The dataset has NaN values.")
    print(new_dataset.isnull().values)
else:
    print("No NaN values found in the dataset.")

Find duplicity values in the merged dataset:

In [ ]:
has_duplicity = new_dataset.duplicated().any()

if has_duplicity:
    print("The dataset has duplicity values.")
    print(new_dataset[new_dataset.duplicated()])
    print("Number of duplicate rows:", new_dataset.duplicated().sum())
else:
    print("No duplicity values found in the dataset.")

Drop values which are not helpful for further training:

In [ ]:
new_dataset.drop('ts', axis=1, inplace=True)
new_dataset.drop('imei', axis=1, inplace=True)

-------------------------------------------------------------------------------------------
### 2.1 B - Data integration
-------------------------------------------------------------------------------------------

#### Standard Deviation 
- detect outliers by standard deviation which spreads data around the mean
- **3x standard deviations ($\sigma$) from the mean ($\mu$)**

In [ ]:
new_dataset.shape

In [ ]:
# Source: https://www.kaggle.com/code/marcinrutecki/outlier-detection-methods

def StandardDevDetection(data, n, columns):
    
    outliers_inx = []
    lower = 0
    upper = 0

    for column in columns:
        # Calculate mean and standard derivation of each column
        data_mean, data_std = mean(data[column], axis=0), std(data[column], axis=0)
        print('column=', column, 'len=', len(data), 'mean=', data_mean, 'std=', data_std)

        # Divide it to the three outliers in the standard deviations:
        cut_off = data_std * 3
        lower, upper = data_mean - cut_off, data_mean + cut_off
        print('column=', column, 'cutoff=', cut_off, 'lower=', lower, 'upper=', upper)

        # Filter the dataframe:
        outliers = data[(data[column] < lower) | (data[column] > upper)].index
        print('Identified outliers:', len(outliers))
        
        outliers_inx.extend(outliers)

    outliers_inx = Counter(outliers_inx)
    multiple_outliers = list( k for k, v in outliers_inx.items() if v > n )

    data_uppper = data[data[column] > upper]
    data_lower = data[data[column] < lower]
    print('Total number of outliers is:', data_uppper.shape[0] + data_lower.shape[0])
    
    return multiple_outliers


columns = new_dataset.columns
result = StandardDevDetection(new_dataset, 1, columns)

new_dataset = new_dataset.drop(result, axis = 0).reset_index(drop=True)

Divide it to the three outliers in the standard deviations:

In [ ]:
new_dataset.shape

Show data distribution after cut of outlines:

In [ ]:
columns = new_dataset.columns

fig, axes = plt.subplots(nrows=(len(columns)//2) - 4, ncols=3, figsize=(30, 30))

axes = axes.flatten()

for i, col in enumerate(columns):
    data_mean, data_std = mean(new_dataset[col], axis=0), std(new_dataset[col], axis=0)
    cut_off = data_std * 3
    lower, upper = data_mean - cut_off, data_mean + cut_off
    sns.histplot(new_dataset[col], kde=True, ax=axes[i])
    axes[i].axvspan(xmin = lower,xmax= new_dataset[col].min(),alpha=0.2, color='red')
    axes[i].axvspan(xmin = upper,xmax= new_dataset[col].max(),alpha=0.2, color='red')
    axes[i].set_title(f'Distribution plot of the column {col}')


-------------------------------------------------------------------------------------------
## 2.1 A - Split dataset on Train and Test data
-------------------------------------------------------------------------------------------

In [ ]:
new_dataset.columns

In [ ]:
target_column = 'mwra'
mwra = new_dataset[target_column]
data = new_dataset.drop(columns=[target_column], axis=1)

In [ ]:
train_data, test_data, train_mwra, test_mwra = train_test_split(data, mwra, test_size=0.3, random_state=42)

-------------------------------------------------------------------------------------------
### 2.1 B Data transformation
-------------------------------------------------------------------------------------------

##### One-hot Encoding Binary Columns

In [ ]:
# Source: IAU week-05

import category_encoders as ce

# create object of BinaryEncoder
ce_binary = ce.BinaryEncoder(cols = ['mwra'])

# fit and transform and you will get the encoded data
ce_binary.fit_transform(train_mwra)

-------------------------------------------------------------------------------------------
### 2.1 C - Scaling
-------------------------------------------------------------------------------------------

#### 2.1 C - Data normalization

## $x_{normalization}=\frac{x-x_{min}}{x_{max} - x_{min}}$

It is good to use, when in the data is a wide range of values. Which can lead to poor performance of models. Which is our case is true. We compared the data and the range was wild.

In [ ]:
# Source: IAU week-05

from numpy import asarray
from sklearn.preprocessing import MinMaxScaler

# define min max scaler
scaler = MinMaxScaler()

# transform data
train_data = scaler.fit_transform(train_data)
print(train_data)

The result is in the numpy python format, which is the best format for training because is fast and intuitive compare to normal array.


#### 2.1 C - Data standardization

## $x_{standardized} = \frac{x -\mu}{\sigma}$
where 
- $\mu$ is the mean  of $x$
- $\sigma$ is the standard deviation of $x$

It is a z-normalization, it measures variations of values about its mean in the dataset. (in short it measures the range of values)


In [ ]:
# Source: IAU week-05

from numpy import asarray
from sklearn.preprocessing import StandardScaler

# define standard scaler
scaler = StandardScaler()

# transform data
train_data = scaler.fit_transform(train_data)
print(train_data)

-------------------------------------------------------------------------------------------
### 2.1 C - Make data distribution more Gaussian :3
-------------------------------------------------------------------------------------------

#### Power transformer 🤖 on random data -> Yeo-Johnson Transform with Linear Regression
- to have more relatively similar to normally distributed

This text is from the week-05:
- Replacing the data with the log, square root, or inverse to remove skew
- Yeo-Johnson transform (default): works with positive and negative values
- Box-Cox transform: only works with strictly positive values
- λ = −1.0 is a reciprocal transform.
- λ = −0.5 is a reciprocal square root transform.  
- λ = 0.0 is a log transform.
- λ = 0.5 is a square root transform.
- λ = 1.0 is no transform.

In [ ]:
# Source: https://www.kaggle.com/code/abhikuks/power-transformers-in-depth-understanding

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PowerTransformer
from matplotlib import pyplot

plt.figure(figsize=(30, 6))

pt = PowerTransformer()
train_data = pt.fit_transform(train_data)
pyplot.hist(train_data, bins=10)

lr = LinearRegression()
lr.fit(train_data, train_mwra)

# let's make some basic prediction on mwra
predictions = lr.predict(train_data)
print(predictions)

# show trained transformed data in the histogram graph of normalized data
plt.hist(train_data, bins=10, alpha=0.7, label='Transformed Data')
plt.legend()
plt.xlabel('Transformed Feature Value')
plt.ylabel('Frequency')
plt.title('Transformed Training Data')
pyplot.hist(train_data, bins=10)

# show prediction
plt.figure(figsize=(10, 6))
plt.scatter(train_mwra, predictions, alpha=0.6)
plt.plot([train_mwra.min(), train_mwra.max()], [train_mwra.min(), train_mwra.max()], 'k--', lw=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs. Predicted Values')


-------------------------------------------------------------------------------------------
#### 2.1 C - Data discretization 
-------------------------------------------------------------------------------------------
- to reduce the effects of minor observation errors
- minimalize the influence of outliers

##### Equal-width dicretization

- We chose this discretization as an example, because the data is still wide range between values and thus this could be a solution to cut range into the smaller intervals. It is called equally sized intervals. We might not use this method in further phases tho- we are unsure of how well the model will behave.

In [ ]:
num_bins = 8

columns = columns.difference(['mwra'])
train_data = pd.DataFrame(train_data, columns=columns)
train_data_binned = pd.DataFrame()

for column in train_data.select_dtypes(include=['float64', 'int64']).columns:
    train_data_binned[f'{column}_binned'] = pd.cut(train_data[column], bins=num_bins, labels=False)

print(train_data_binned.head())

fig, axes = plt.subplots(nrows=(len(columns)//2) - 5, ncols=3,figsize=(30, 50))
axes = axes.flatten()
i = 0
for col in train_data_binned:
    sns.histplot(train_data_binned[col], kde=True, ax=axes[i], color='blue', edgecolor='black')
    axes[i].set_title(f'Equal-Width Binned Data for {col}')
    i+= 1
plt.xlabel('Bins')
plt.ylabel('Frequency')

-------------------------------------------------------------------------------------------
### 2.1 D - Reasons for implementation
-------------------------------------------------------------------------------------------

Some things are already indicated in the text. However, we decided to merge processes and connections into one dataset since they have common columns and values. They were merged on ts, imei and mwra, these columns they had absolutely identical. 
In the dataset, there are not anymore ts and imei columns, because they were not adequate for testing purposes and we considered them as unnecessary. 

Furthermore, we renamed the columns to better names for reading and better orientation during work. For example, the katana represents Facebook. Why not change it to right away? 

After merging for sure, we checked if the data are clean enough for further work.  

In our opinion, it is better to cut outliers before splitting the dataset. We also checked the presentation, and it was recommended to do it this way. After that, we split the dataset to train and test into two groups one is for all data and the second is for predicted value mwra. This way it is possible to predict how the mwra will behave. 

In conclusion, we prepared data for machine learning by data cleaning (missing values, outliers detection), integration (3x standard deviation, encoding) and transformation (scaling, transformers, discretization). Which ways we picked it are already defined in the specific sections.

--------------------------------

--------------------------------
## 2.2 Attribute selection
--------------------------------

### 2.2 A - Comparing feature selection methods

### Filter Method

As the first filter attribute selection method, we decided against using variance threshold method, as it did nothing. Instead we will use Mutual Information.

**Mutual Information (MI)** is a measure of the mutual dependence between two variables. It quantifies how much information knowing one variable reduces the uncertainty about the other. In feature selection, mutual information tells us how informative a feature is about the target variable, regardless of whether the relationship is linear or non-linear.

From Lab study material file- week-05:
The concept of $MI$ is linked to information theory and **information entropy** ($\mathcal{H}$). The unit of information depends on the base of the logarithm. If the base is 2, the most used, then the information is measured in *bits*. **MI is non-negative and symmetric.**

#### $\mathcal{H}(X) = - \int dx~\mu(x)~log\mu(x)$
#### $I(X, Y) = - \int \int dx~dy~\mu(x, y)~log \frac{\mu(x,y)}{\mu_x(x)~\mu_y(y)}$

For discrete variables,  $H(X)$ is calculated as:
#### $H(X) = -\sum_i P(x_{i}) log P(x_{i})$

MI can be equivalently expressed as the amount of uncertainty in $X$ minus the amount of uncertainty in $X$ after $Y$ is known, denoted as

#### $I(X; Y)= H(X)- H(X|Y)$

The entropy of $X$ after observing values of $Y$ is formulated by
#### $H(X|Y) = -\sum_{\substack{j}} P(y_{j}) \sum_{\substack{i}}P(x_{i}|y_{j})\log_2{P(x_{i}|y_{j})}$

where 
- $P(x_i)$ is the prior probabilities for all values of $X$ and 
- $P(x_i|y_i)$ is the posterior probabilities of $X$ given the values of $Y$. 

<sub><sup>(Adam note: Ngl plne nechapem tymto matematickym hieroglyfom)</sup></sub>


In [ ]:
from sklearn.feature_selection import mutual_info_classif
num_of_features = 10

selector = mutual_info_classif(train_data, train_mwra)
scores = pd.Series(selector, index=train_data.columns).sort_values(ascending=False)
mi_selected_features = scores[:num_of_features].index

print("The top " + num_of_features.__str__() + " features according to F-value method are: ")
print(mi_selected_features)


As the second filter method, we decided to use **F-value** method since our data is continuous.

**F-value** source: Lab study material file- week-05

The correlation between each regressor and the target is computed by

### $F_{value} = \frac{variance_{dataset_1}}{variance_{dataset_2}} = \frac{\sigma_1^2}{\sigma_2^2}$

It is converted to an F score then to a p-value.

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
num_of_features = 10

selector = SelectKBest(score_func=f_classif, k=num_of_features)
selector.fit(train_data, train_mwra)

# get scores and p-values
f_scores = pd.Series(selector.scores_, index=train_data.columns)
f_scores_sorted = f_scores.sort_values(ascending=False)
f_selected_features = f_scores_sorted.index[:num_of_features]

print(f"\nThe top {num_of_features} features according to F-value method are:")
print(f_selected_features)

### Wrapper

These methods evaluate subsets of features by actually training a model, making them more accurate for specific algorithms.

For showcasing wrapper method we chose RFE (Recursive feature elimination)

RFE is a wrapper method that uses the built-in importance of the estimator attributes (in this case **RandomForest**) to generate the ratings. RFE is able to account for interactions between attributes, but is computationally more demanding than filter methods (note: look at the execution time of this cell lol).

In [ ]:
from sklearn.feature_selection import SelectKBest, RFE
from sklearn.ensemble import RandomForestClassifier
num_of_features = 10
model = RandomForestClassifier()

#rfe
rfe = RFE(estimator=model, n_features_to_select=num_of_features)
tmp = rfe.fit_transform(train_data, train_mwra)

rfe_selected_features = rfe.get_support(indices=True)

rfe_selected_features = train_data.columns[rfe_selected_features]

final_rf = RandomForestClassifier()
final_rf.fit(train_data[rfe_selected_features], train_mwra)

# get and sort features by importance
rf_importance = pd.Series(final_rf.feature_importances_, index=rfe_selected_features)
rfe_selected_features = rf_importance.sort_values(ascending=False).index

print(f"\nThe top {num_of_features} features according to RFE method are:")
print(rfe_selected_features)

So to recap everything...

In [ ]:
num_of_features = 10

print("Columns selected by every method: ")
mi_set = set(mi_selected_features)
f_set = set(f_selected_features)
rfe_set = set(rfe_selected_features)

common_features = mi_set.intersection(f_set, rfe_set)
print("\nFeatures selected by all three methods:")
for feature in common_features:
    print(f"- {feature}")

comparison_df = pd.DataFrame({
    'Feature': list(set(list(mi_selected_features) + list(f_selected_features) + list(rfe_selected_features))),
    'Mutual Information': False,
    'F-Value': False,
    'RFE': False
})

comparison_df.loc[comparison_df['Feature'].isin(mi_selected_features), 'Mutual Information'] = True
comparison_df.loc[comparison_df['Feature'].isin(f_selected_features), 'F-Value'] = True
comparison_df.loc[comparison_df['Feature'].isin(rfe_selected_features), 'RFE'] = True

################
# help with stylization by Claude.ai
################

def color_boolean(val):
    if isinstance(val, bool):
        color = '#90EE90' if val else '#FFB6C1'  # Light green if True, light red if False
        return f'background-color: {color}'
    return ''

# Apply styling to the DataFrame
styled_df = (comparison_df.style
             .map(color_boolean, subset=['Mutual Information', 'F-Value', 'RFE'])
             .set_properties(**{
    'text-align': 'center',
    'border': '1px solid gray',
    'padding': '8px'
})
             .set_properties(subset=['Feature'], **{
    'text-align': 'left',
    'font-weight': 'bold',
    'background-color': '#F0F8FF'  # Light blue background for feature names
})
             .set_table_styles([
    {'selector': 'th',
     'props': [('background-color', '#4682B4'),  # Steel blue headers
               ('color', 'white'),
               ('font-weight', 'bold'),
               ('text-align', 'center'),
               ('padding', '8px'),
               ('border', '1px solid gray')]},
    {'selector': 'caption',
     'props': [('caption-side', 'top'),
               ('font-size', '16px'),
               ('font-weight', 'bold'),
               ('color', '#2F4F4F'),  # Dark slate gray
               ('padding', '8px')]}
])
             .set_caption('Feature Selection Methods Comparison'))

# Display the styled DataFrame
display(styled_df)

# sets of features unique to each method
mi_unique = mi_set - (f_set | rfe_set)
f_unique = f_set - (mi_set | rfe_set)
rfe_unique = rfe_set - (mi_set | f_set)

print("\nFeatures unique to each method:")
print("Mutual Information only:", mi_unique if mi_unique else "None")
print("F-Value only:", f_unique if f_unique else "None")
print("RFE only:", rfe_unique if rfe_unique else "None")

################
# source: Claude.ia
################

def create_ranking_comparison(mi_features, f_features, rfe_features):
    # Create a dictionary to store rankings
    rankings = {}

    # Add rankings for each method (1 being highest importance)
    for i, feature in enumerate(mi_features):
        if feature not in rankings:
            rankings[feature] = {'MI_rank': i + 1}
        else:
            rankings[feature]['MI_rank'] = i + 1

    for i, feature in enumerate(f_features):
        if feature not in rankings:
            rankings[feature] = {'F_rank': i + 1}
        else:
            rankings[feature]['F_rank'] = i + 1

    for i, feature in enumerate(rfe_features):
        if feature not in rankings:
            rankings[feature] = {'RFE_rank': i + 1}
        else:
            rankings[feature]['RFE_rank'] = i + 1

    # Convert to DataFrame
    ranking_df = pd.DataFrame.from_dict(rankings, orient='index')

    # Fill NaN with 0 (features not selected by a method)
    ranking_df = ranking_df.fillna(0)

    # Calculate average rank (excluding zeros)
    ranking_df['Avg_Rank'] = ranking_df.replace(0, np.nan).mean(axis=1)

    # Count methods that selected each feature
    ranking_df['Methods_Count'] = (ranking_df[['MI_rank', 'F_rank', 'RFE_rank']] != 0).sum(axis=1)

    # Sort by Methods_Count (descending) and then by Avg_Rank (ascending)
    ranking_df = ranking_df.sort_values(['Methods_Count', 'Avg_Rank'], ascending=[False, True])

    return ranking_df

# Create the ranking comparison
ranking_comparison = create_ranking_comparison(
    mi_selected_features,
    f_selected_features,
    rfe_selected_features
)

ranking_comparison

From our observations, we can see that Mutual Information and F-value selected the same features. There is also a big overlap with RFE that selected eight of the features that the other two methods selected.  While MI and F-value selected facebook.y and inputmethod.latin RFE selected UCMobile.x86 and google- clearly showing that these features have more complex relationships with mwra.

Furthermore, the one feature consistently considered as the most important is gmail_x followed by gapps, facebook_x and chrome_x.

-------------------------------------------------------------------------------------------
## Phase 3
-------------------------------------------------------------------------------------------
### 3.1 Simple classification based on dependencies in data

First, let's import our implementation of ID3 and data preprocessing pipeline.

In [2]:
import import_ipynb
import id3_implementation as id3
import preprocessing as pp

No NaN values found in the dataset.
The dataset has duplicity values.
                        ts                 imei  mwra  facebook_x  chrome_x  \
95     1525520040000000000  8630330696303482303   1.0    10.82916  11.29582   
108    1525520760000000000   863033069630348065   1.0    14.71992  14.01692   
175    1525524720000000000   359043379931766353   0.0     8.46728  13.41079   
300    1525532160000000000  8630330696303481669   1.0     9.87679   8.52849   
303    1525532280000000000   359043379931766924   1.0    10.22849  11.46160   
...                    ...                  ...   ...         ...       ...   
15445  1525987200000000000  3590433799317661925   1.0    11.23638  12.54494   
15446  1526140320000000000   863033069630348776   0.0    11.71795  13.86245   
15447  1526140320000000000   863033069630348776   0.0    11.71795  13.86245   
15448  1526338140000000000  3590433799317661206   1.0    15.51197  16.53309   
15449  1526338140000000000  3590433799317661206   1.0    15.5

Now, let's copy the preprocessed data from the notebook and build the tree.

**BIG NOTE**: This little maneuver is gonna cost us 51 years (or roughly 7 minutes)

<sub><sup>(Shout out to everyone who got the Interstellar ref.)</sup></sub>


In [3]:
# Copy and convert to np types the preprocessed data for purposes of training the ID3 tree
# (Adam Note: We don't need to copy the data, we can still just read the csv files we created- I was not sure which
# one to do so...)

y_train = pp.y_train_df.to_numpy()
X_train =  pp.X_train_processed_df.to_numpy()
y_test = pp.y_test_df.to_numpy()
X_test = pp.X_test_processed_df.to_numpy()

In [17]:
# Train the ID3 tree (without a depth limit)
id3_tree = id3.build_tree(X_train, y_train.flatten(), -1) # flatten - bc the function expects 1D array for y

### 3.1 B Evaluation of the model

Now, let's evaluate the model on the test data with metrics like accuracy, precision and recall.

In [18]:
# make predictions
y_prediction = []
for i in range(len(X_test)):
    y_prediction.append(id3.predict(id3_tree, X_test[i]))

from sklearn.metrics import accuracy_score, precision_score, recall_score
id3_accuracy = accuracy_score(y_test, y_prediction)
print(f"Accuracy: {id3_accuracy:.4f}")

id3_precision = precision_score(y_test, y_prediction, average='weighted')
print(f"Precision: {id3_precision:.4f}")

id3_recall = recall_score(y_test, y_prediction, average='weighted')
print(f"Recall: {id3_recall:.4f}")

Accuracy: 0.7777
Precision: 0.7779
Recall: 0.7777


### 3.1 C Overfitting

Now, let's look at overfitting. We will evaluate the model on the training data.

Overfitting is commonly detected by comparing the accuracy of the model on the training data and the test data. If the model has a higher accuracy on the training data, it is likely overfitting.

In [ ]:
# check for overfitting
# create predictions for the training data
y_train_prediction = []
for i in range(len(X_train)):
    y_train_prediction.append(id3.predict(id3_tree, X_train[i]))

id3_train_accuracy = accuracy_score(y_train, y_train_prediction)
print(f"Training Accuracy: {id3_train_accuracy:.2f}")

# Check for overfitting
if id3_train_accuracy > id3_accuracy:
    print("The model is overfitting.")
else:
    print("The model is not overfitting.")

Here the accuracy with training data for this model is 1.0, which is a clear sign of overfitting. The model is too complex and it is not generalizing well to new data.

To be honest, after writing this, we realized that we can improve the overfitting by limiting the depth of the tree. We will improve our implementation and re-run the test.

Now we will try different possible limits with this code.

In [ ]:
# Train the ID3 tree with a depth limit
id3_tree_depth_limited = id3.build_tree(X_train, y_train.flatten(), 9)

# make predictions
y_prediction_depth_limited = []
for i in range(len(X_test)):
    y_prediction_depth_limited.append(id3.predict(id3_tree_depth_limited, X_test[i]))

# evaluate the model
id3_accuracy_depth_limited = accuracy_score(y_test, y_prediction_depth_limited)
print(f"Accuracy: {id3_accuracy_depth_limited:.4f}")

id3_precision_depth_limited = precision_score(y_test, y_prediction_depth_limited, average='weighted')
print(f"Precision: {id3_precision_depth_limited:.4f}")

id3_recall_depth_limited = recall_score(y_test, y_prediction_depth_limited, average='weighted')
print(f"Recall: {id3_recall_depth_limited:.4f}")

# create predictions for the training data
y_train_prediction_depth_limited = []
for i in range(len(X_train)):
    y_train_prediction_depth_limited.append(id3.predict(id3_tree_depth_limited, X_train[i]))

id3_train_accuracy_depth_limited = accuracy_score(y_train, y_train_prediction_depth_limited)
print(f"Training Accuracy: {id3_train_accuracy_depth_limited:.2f}")

# Check for overfitting (found this was not correct...)
# if id3_train_accuracy_depth_limited > id3_accuracy_depth_limited:
#     print("The model is overfitting.")
# else:
#     print("The model is not overfitting.")

Results after limiting the depth of the tree to **5**:
- Accuracy: 0.8120
- Precision: 0.8127
- Recall: 0.8120
- Training Accuracy: 0.83

The accuracy is much better than without the depth limit. Let's try to limit the depth even further.

Results after limiting the depth of the tree to **4**:
- Accuracy: 0.8053
- Precision: 0.8071
- Recall: 0.8053
- Training Accuracy: 0.82

Now we see that the accuracy is slightly worse, but the model might be underfitting (after further testing, it seems like the tree is underfitted). Now let's try to limit the depth to 6.

Results after limiting the depth of the tree to **6**:
- Accuracy: 0.8356
- Precision: 0.8353
- Recall: 0.8356
- Training Accuracy: 0.86

The accuracy is better than with the depth limit of 5. Let's try limit 7.

Results after limiting the depth of the tree to **7**:
- Accuracy: 0.8374
- Precision: 0.8373
- Recall: 0.8374
- Training Accuracy: 0.87

Results after limiting the depth of the tree to **9**:
- Accuracy: 0.8407
- Precision: 0.8407
- Recall: 0.8407
- Training Accuracy: 0.89

The accuracy is even better!

(Adam: I went on a bit of a journey with this...)

At this point, we can just generate and plot the metrics for different depths, even though it's beyond the scope of the assignment.

NOTE: We turned this cell into a "raw" cell to avoid running it by accident. (Run time was roughly 1,5 hours)

Results: ![ID3 - depth testing](images/ID3_depth_testing.png "id3_depth_testing")

With this testing we can see that this model can get easily and extremely overfitted. The accuracy is the best with depth 9.

-------------------------------------------------------------------------------------------
### 3.2 Training and Evaluation of Machine Learning Classifiers
-------------------------------------------------------------------------------------------

### 3.2 A - Tree algorithm from scikit-learn

While with ID3 we show pre-pruning by limiting the depth of the three. We will now use post-pruning while showcasing the use of DecisionTreeClassifier from scikit-learn to train a tree model.

### Pruning

For post-pruning, we will use [Minimal Cost-Complexity Pruning](https://scikit-learn.org/1.5/modules/tree.html#minimal-cost-complexity-pruning) (MCCP). This method prunes the tree by removing nodes that have a cost complexity less than a threshold $\alpha$.

### DecisionTreeClassifier or RandomTreeRegressor
Now, considering which three from scikit-learn to use — weather DecisionTreeClassifier or RandomTreeRegressor. Since our problem is a classification problem (predicting mwra), we decided to use DecisionTreeClassifier. 

### Gini Impurity
For criteria, we chose gini, which is the default one. Gini impurity is an alternative metric to entropy that is also used to measure the quality of a split in decision tree classification algorithms. Like entropy, Gini impurity quantifies the level of impurity or uncertainty in a dataset, but it does so in a slightly different way.

Gini impurity, denoted as G(X), is defined as the probability of a randomly chosen element being incorrectly classified if it was randomly labeled according to the distribution of labels in the dataset. Mathematically, it is calculated as:
$$G(X) = 1 - Σ p(x)^2$$
Where p(x) is the probability mass function of the random variable X (the class labels).

Let's train the model and evaluate it without any pruning first.

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Train the DecisionTreeClassifier
dtc = DecisionTreeClassifier(criterion='gini', random_state=42)
dtc.fit(X_train, y_train)

# make predictions
y_prediction_dtc = dtc.predict(X_test)

dtc_accuracy = accuracy_score(y_test, y_prediction_dtc)
print(f"Accuracy: {dtc_accuracy:.4f}")

dtc_precision = precision_score(y_test, y_prediction_dtc, average='weighted')
print(f"Precision: {dtc_precision:.4f}")

dtc_recall = recall_score(y_test, y_prediction_dtc, average='weighted')
print(f"Recall: {dtc_recall:.4f}")

Accuracy: 0.7741
Precision: 0.7738
Recall: 0.7741


The results are slightly worse than with ID3 which is surprising. Now, let's try to prune the tree using MCCP.

In [24]:
# Train the DecisionTreeClassifier with pruning
dtc_pruned = DecisionTreeClassifier(criterion='gini', random_state=42, ccp_alpha=0.001)
dtc_pruned.fit(X_train, y_train)

# make predictions
y_prediction_dtc = dtc_pruned.predict(X_test)

dtc_accuracy = accuracy_score(y_test, y_prediction_dtc)
print(f"Accuracy: {dtc_accuracy:.4f}")

dtc_precision = precision_score(y_test, y_prediction_dtc, average='weighted')
print(f"Precision: {dtc_precision:.4f}")

dtc_recall = recall_score(y_test, y_prediction_dtc, average='weighted')
print(f"Recall: {dtc_recall:.4f}")


Accuracy: 0.8385
Precision: 0.8388
Recall: 0.8385


The results are slightly better than without pruning. 

### Finding the best $\alpha$

Now, let's try to find the best $\alpha$ value for pruning. We will use cross-validation to find the best $\alpha$ value. (We know it's too early and it is required further in the assignment, but we couldn't help it)

In [27]:
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

path = dtc.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas = path.ccp_alphas[:-1]  # minus the last value as it is the root node

dtcs = []

# go through each possible alpha value from the path
for ccp_alpha in ccp_alphas:
    dtc = DecisionTreeClassifier(random_state=42, ccp_alpha=ccp_alpha)
    dtc.fit(X_train, y_train)
    dtcs.append(dtc)

# use of cross-validation
train_scores = [cross_val_score(dtc, X_train, y_train, cv=5).mean() for dtc in dtcs]

best_alpha_index = train_scores.index(max(train_scores))
best_alpha = ccp_alphas[best_alpha_index]
print(f"Best alpha: {best_alpha:.4f} with Cross-validation Accuracy: {train_scores[best_alpha_index]:.4f}")

# Train final model with best alpha
best_model = DecisionTreeClassifier(random_state=42, ccp_alpha=best_alpha)
best_model.fit(X_train, y_train)

Best alpha: 0.0007 with Cross-validation Accuracy: 0.8509


DecisionTreeClassifier(ccp_alpha=np.float64(0.0006913108988314022),
                       random_state=42)